In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
import scipy.optimize as optimize
import warnings
warnings.filterwarnings("ignore")

In [2]:
SF_est = pd.read_excel('delphi-consensus-outputs.xlsx')
MR_sf = pd.read_excel('margin-revenue-salesforce.xlsx')
drugs = np.array(SF_est.columns[1:])
drugs

array(['Naprosyn', 'Nipro', 'Anaprox', 'Norinyl', 'Pironil', 'Lidex',
       'Synalar', 'Nasalide'], dtype=object)

In [3]:
SF_est = SF_est.drop(SF_est.columns[[0]], axis=1)
delphi_data = np.array(SF_est)
delphi_data

array([[0.3 , 0.47, 0.15, 0.31, 0.45, 0.56, 0.59, 0.15],
       [0.45, 0.68, 0.48, 0.63, 0.7 , 0.8 , 0.76, 0.61],
       [1.  , 1.05, 1.04, 1.03, 1.01, 1.02, 1.02, 1.07],
       [1.4 , 1.26, 1.2 , 1.15, 1.05, 1.11, 1.07, 1.46],
       [1.6 , 1.52, 1.35, 1.25, 1.1 , 1.2 , 1.11, 1.76]])

In [4]:
MR_sf = MR_sf.drop(MR_sf.columns[[0]], axis=1)
mar_rev_sf = np.array(MR_sf)
mar_rev_sf

array([[  0.7 ,   0.8 ,   0.55,   0.72,   0.72,   0.62,   0.53,   0.52],
       [214.4 , 210.1 ,  36.5 ,  21.2 ,  37.2 ,  38.  ,  14.6 ,  11.2 ],
       [ 96.8 ,  97.2 , 142.4 ,  52.7 ,  24.1 ,  27.3 ,  29.7 ,  56.8 ]])

## Part (A)

In [5]:
proxy_for_infinity = 10
x = [0, 0.5, 1.0, 1.5, proxy_for_infinity]

In [6]:
def sales_response(x, c, d, adbudg_min, adbudg_max):
    return adbudg_min + ((adbudg_max - adbudg_min)*(x**c)/(d + x**c))

params = pd.DataFrame(columns=['Drugs', 'c', 'd', 'adbudg_min', 'adbudg_max'])
for i in range(8):
    y = delphi_data[:,i]
    y_lowest = y[0] 
    y_highest = y[4]
    popt, pcov = curve_fit(sales_response, xdata = x, ydata = y, p0=[1, 1, y_lowest, y_highest])
    res = {'Drugs': drugs[i], 'c': round(popt[0], 3), 'd': round(popt[1], 3), 'adbudg_min': round(popt[2], 3), 'adbudg_max': round(popt[3], 3)}
    params = params.append(res, ignore_index=True)
params

,Drugs,c,d,adbudg_min,adbudg_max
0,Naprosyn,3.463,0.842,0.309,1.607
1,Nipro,2.264,0.826,0.469,1.524
2,Anaprox,2.817,0.361,0.148,1.343
3,Norinyl,2.618,0.312,0.310,1.248
4,Pironil,3.254,0.162,0.450,1.092
5,Lidex,2.089,0.394,0.560,1.202
6,Synalar,3.246,0.212,0.590,1.106
7,Nasalide,2.045,0.677,0.157,1.781


## Part (B)

In [7]:
prof_margin = mar_rev_sf[0,:]
og_revenues = mar_rev_sf[1,:]
og_salesf = mar_rev_sf[2,:]
cost_sf = 0.057

def negative_profit(sf, c, d, ad_min, ad_max, og_sf, rev, mar, cost_sf = 0.057):
    sf_rel = sf / og_sf
    response = ad_min + ((ad_max - ad_min)*(sf_rel**c)/(d + sf_rel**c))
    profit = (response * rev * mar) - (sf * cost_sf)
    return -1 * profit

In [8]:
params = params.drop(params.columns[[0]], axis=1)
param = np.array(params)

In [9]:
lower_bound = 0
x0 = 80

sf_profit = pd.DataFrame(columns=['Drugs', 'Salesforce Size', 'Profit'])
for i in range(8):
    mar = prof_margin[i]
    og_sf = og_salesf[i]
    rev = og_revenues[i]
    c, d, ad_min, ad_max = param[i,:]
    bounds_object = optimize.Bounds(lower_bound, np.inf)
    optimizer_output = optimize.minimize(negative_profit, x0, args=(c, d, ad_min, ad_max, og_sf, rev, mar, cost_sf), method='trust-constr', bounds=bounds_object, options={'verbose': 1})
    salesforce_size = optimizer_output['x'][0]
    opt_profit = -1 * optimizer_output['fun']
    res = {'Drugs': drugs[i],'Salesforce Size': round(salesforce_size, 2), 'Profit': round(opt_profit, 3)}
    sf_profit = sf_profit.append(res, ignore_index=True)
    # print(optimizer_output)

`gtol` termination condition is satisfied.
Number of iterations: 21, function evaluations: 26, CG iterations: 12, optimality: 8.00e-09, constraint violation: 0.00e+00, execution time: 0.049 s.
`gtol` termination condition is satisfied.
Number of iterations: 21, function evaluations: 26, CG iterations: 12, optimality: 6.55e-09, constraint violation: 0.00e+00, execution time: 0.045 s.
`gtol` termination condition is satisfied.
Number of iterations: 18, function evaluations: 20, CG iterations: 9, optimality: 1.98e-09, constraint violation: 0.00e+00, execution time: 0.037 s.
`gtol` termination condition is satisfied.
Number of iterations: 15, function evaluations: 14, CG iterations: 6, optimality: 3.10e-09, constraint violation: 0.00e+00, execution time: 0.03 s.
`gtol` termination condition is satisfied.
Number of iterations: 23, function evaluations: 30, CG iterations: 14, optimality: 6.94e-09, constraint violation: 0.00e+00, execution time: 0.04 s.
`gtol` termination condition is satisfi

In [10]:
sf_profit

,Drugs,Salesforce Size,Profit
0,Naprosyn,270.53,221.199
1,Nipro,330.11,228.595
2,Anaprox,171.69,12.960
3,Norinyl,71.33,13.212
4,Pironil,36.86,26.475
5,Lidex,49.98,23.954
6,Synalar,30.65,6.170
7,Nasalide,71.16,3.486


In [11]:
sf_profit = sf_profit.drop(sf_profit.columns[[0]], axis=1)
sf_profit_b = np.array(sf_profit)

## Part (C)

In [12]:
def negative_profit_c(sf, c, d, ad_min, ad_max, og_sf, rev, mar, cost_sf = 0.057):
    sf_rel = sf / og_sf
    response = ad_min + ((ad_max - ad_min)*(sf_rel**c)/(d + sf_rel**c))
    profit = (response * rev * mar) - (sf * cost_sf)
    sumprofit = np.sum(profit)
    return -1 * sumprofit

In [13]:
n_drugs = 8
total_salesforce_size = 700
lower_bound = 0

c = param[:,0]
d = param[:,1]
ad_min = param[:,2]
ad_max = param[:,3]

mar = mar_rev_sf[0,:]
rev = mar_rev_sf[1,:]
og_sf = mar_rev_sf[2,:]

x0c = np.ones(n_drugs)*total_salesforce_size/n_drugs
sum_constraint_object = optimize.LinearConstraint(np.ones((1, n_drugs)), lower_bound, total_salesforce_size)
bounds_object_c = optimize.Bounds(lower_bound, np.inf)

optimizer_output_c = optimize.minimize(negative_profit_c, x0c, args=(c, d, ad_min, ad_max, og_sf, rev, mar, cost_sf), method='SLSQP', bounds=bounds_object_c, constraints=sum_constraint_object)
salesforce_size = optimizer_output_c['x']
opt_profit = -1 * optimizer_output_c['fun']
sf_rel = salesforce_size / og_sf
response = ad_min + ((ad_max - ad_min)*(sf_rel**c)/(d + sf_rel**c))
profit = ((response * rev * mar) - (salesforce_size * cost_sf))
profit_reduction = (profit - sf_profit_b[:,1])/sf_profit_b[:,1]
sf_reduction = (salesforce_size - sf_profit_b[:,0])/sf_profit_b[:,0]

In [14]:
opt_prof = pd.DataFrame(columns=['Drugs', 'Salesforce', 'Profit', 'Profit Reduction', 'Salesforce Reduction'])
for i in range(8):
    rows = {'Drugs': drugs[i], 'Salesforce': round(salesforce_size[i], 2), 'Profit': round(profit[i], 3), 'Profit Reduction': round(profit_reduction[i], 3), 'Salesforce Reduction': round(sf_reduction[i],3)}
    opt_prof = opt_prof.append(rows, ignore_index=True)
opt_prof

,Drugs,Salesforce,Profit,Profit Reduction,Salesforce Reduction
0,Naprosyn,220.27,219.553,-0.007,-0.186
1,Nipro,245.95,225.873,-0.012,-0.255
2,Anaprox,117.47,11.076,-0.145,-0.316
3,Norinyl,51.52,12.552,-0.050,-0.278
4,Pironil,29.47,26.233,-0.009,-0.200
5,Lidex,35.33,23.478,-0.020,-0.293
6,Synalar,0.00,4.565,-0.260,-1.000
7,Nasalide,0.00,0.914,-0.738,-1.000
